In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [24]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

import argparse
import ast
import matplotlib.pyplot as plt
import logging
import sys  
sys.path.insert(0, '../lib')
from utils import *
logging.basicConfig(level=logging.INFO)

In [40]:
%run ../lib/mple_learn_lib.ipynb
%run ../lib/report_lib.ipynb
%run ../lib/make_data_lib.ipynb

# ergm1

* edges + triangle + transitiveties
* n: 20
* T: 30
* n cpt: 5

In [435]:
H_outfile

'../../output/ergm1/sim1000.rds_H.txt'

In [38]:
%%time
conf = {
    "outdir": "../../output/ergm1/results4",
    "data_name": "sim1000.rds",
#     "form_terms": ['edges'],
#     "diss_terms": ['edges'],
#     "form_terms": ['edges', 'triangle'],
#     "diss_terms": ['edges', 'triangle'],
#     "form_terms": ['edges', 'triangle', 'transitiveties'],
#     "diss_terms": ['edges', 'triangle', 'transitiveties'],
    "form_terms": ['edges', 'transitiveties'],
    "diss_terms": ['edges', 'transitiveties'],
    "true_change_points": [ 19, 39, 59, 79],
}

input_data = conf["data_name"] + ('' if conf["data_name"].endswith('.rds') else '.rds')
data = readRDS("../../data/ergm/" + input_data)

output_dir = conf["outdir"] + ('' if conf["outdir"].endswith('/') else '/')
H_outfile = output_dir + conf["data_name"] + "_H.txt"
y_outfile = output_dir + conf["data_name"] + "_y.txt"


args_dir = utils.make_directory(output_dir, 'args')
utils.save_args(conf, args_dir + "args.txt")
save_H_y(data[0], conf, H_outfile, y_outfile)

time series length: 100
network size: 30 x 30
statistics dimension: 4
form terms are: ['edges', 'transitiveties']
diss terms are: ['edges', 'transitiveties']
Saving change statistics...
shape of H: (100, 900, 4)
shape of y: (100, 900)
CPU times: user 1.91 s, sys: 41 ms, total: 1.96 s
Wall time: 1.96 s


In [538]:
# args = {
#     'inputdir': "../../output/ergm3",
#     "result_identifier": 1,
# }

H, y, experiment_dir, sim_args = load_data(conf)
# result_dir = make_directory(experiment_dir, 'results')
# args_dir = make_directory(result_dir, 'args' + str(args["result_identifier"]))
# save_args(args, args_dir + 'args_model.txt')
theta_outfile = experiment_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = experiment_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = experiment_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = experiment_dir + 'est_theta_diff_' + sim_args["data_name"][:-4] + '.png'

INFO:root:Loading network statistics and edge data...


{"outdir": "../../output/ergm1/results4", "data_name": "sim1000.rds", "form_terms": ["edges", "transitiveties"], "diss_terms": ["edges", "transitiveties"], "true_change_points": [19, 39, 59, 79]}



INFO:root:Data has dimension (t, n, p): (100, 30, 4)


In [539]:
H.max()

29.0

In [541]:
conf = {
    'lam': 2,
    'admm_alpha': 100,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 100,
    'newton_max_steps': 100,
    'converge_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
    "verbose": 1,
}

In [542]:
%%time
model = STERGMGraph(
    **conf,
)

model.load_H_y(H, y)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(
    solver=conf["solver"],
#     initial_values={
#         "theta": np.tile(np.array([-1.111, -1.982, -2.057, 2.590]), (model.t,1)),
#         "u": np.zeros((model.t, model.p)),
#         "z": np.random.normal(loc=0, scale=0.2, size=(model.t, model.p)),
#     }
)
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 0.26069092750549316.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.009970903396606445.
[INFO] Updating u...
[INFO] max mu :  0.9999851549334124
[INFO] dual_resnorm: 0.475938
[INFO] primal_resnorm: 0.357822
[INFO] convergence: 0.475938
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 0.25413990020751953.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.015048027038574219.
[INFO] Updating u...
[INFO] max mu :  0.9999899862498585
[INFO] dual_resnorm: 0.280600
[INFO] primal_resnorm: 0.177430
[INFO] convergence: 0.280600
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 0.21432232856750488.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.016254901885986328.
[INFO] Updating u...
[INFO] max mu :  0.999992

[INFO] Newton converged in: 0.16558289527893066.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.014086008071899414.
[INFO] Updating u...
[INFO] max mu :  0.9999754382662985
[INFO] dual_resnorm: 0.003609
[INFO] primal_resnorm: 0.000444
[INFO] convergence: 0.003609
-------------------------------------------------
[INFO] ADMM step #24
[INFO] Updating theta...
[INFO] Newton converged in: 0.1594831943511963.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.012955188751220703.
[INFO] Updating u...
[INFO] max mu :  0.9999748238582897
[INFO] dual_resnorm: 0.003313
[INFO] primal_resnorm: 0.000394
[INFO] convergence: 0.003313
-------------------------------------------------
[INFO] ADMM step #25
[INFO] Updating theta...
[INFO] Newton converged in: 0.16068816184997559.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.013061761856079102.
[INFO] Updating u...
[INFO] max mu :  0.9999742430733957
[INFO] dual_resnorm: 0.003050
[INFO] primal_resnorm: 0.000351
[

[INFO] Newton converged in: 0.2714390754699707.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.12536907196044922.
[INFO] Updating u...
[INFO] max mu :  0.9999997476425526
[INFO] dual_resnorm: 0.005692
[INFO] primal_resnorm: 0.002814
[INFO] convergence: 0.005692
-------------------------------------------------
[INFO] ADMM step #46
[INFO] Updating theta...
[INFO] Newton converged in: 0.272017240524292.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.1260688304901123.
[INFO] Updating u...
[INFO] max mu :  0.9999997631653893
[INFO] dual_resnorm: 0.005073
[INFO] primal_resnorm: 0.002558
[INFO] convergence: 0.005073
-------------------------------------------------
[INFO] ADMM step #47
[INFO] Updating theta...
[INFO] Newton converged in: 0.2631568908691406.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.12682485580444336.
[INFO] Updating u...
[INFO] max mu :  0.999999775748949
[INFO] dual_resnorm: 0.004533
[INFO] primal_resnorm: 0.002326
[INFO] co

[INFO] Newton converged in: 0.2108321189880371.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.14665985107421875.
[INFO] Updating u...
[INFO] max mu :  0.999999841002243
[INFO] dual_resnorm: 0.000883
[INFO] primal_resnorm: 0.000261
[INFO] convergence: 0.000883
-------------------------------------------------
[INFO] ADMM step #68
[INFO] Updating theta...
[INFO] Newton converged in: 0.2012321949005127.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.14565205574035645.
[INFO] Updating u...
[INFO] max mu :  0.9999998414841496
[INFO] dual_resnorm: 0.000838
[INFO] primal_resnorm: 0.000233
[INFO] convergence: 0.000838
-------------------------------------------------
[INFO] ADMM step #69
[INFO] Updating theta...
[INFO] Newton converged in: 0.19804596900939941.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.14551401138305664.
[INFO] Updating u...
[INFO] max mu :  0.99999984190094
[INFO] dual_resnorm: 0.000797
[INFO] primal_resnorm: 0.000208
[INFO] c

[INFO] Newton converged in: 0.07868814468383789.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.14808082580566406.
[INFO] Updating u...
[INFO] max mu :  0.9999998442145047
[INFO] dual_resnorm: 0.000338
[INFO] primal_resnorm: 0.000038
[INFO] convergence: 0.000338
-------------------------------------------------
[INFO] ADMM step #90
[INFO] Updating theta...
[INFO] Newton converged in: 0.09251999855041504.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.14847683906555176.
[INFO] Updating u...
[INFO] max mu :  0.9999998442180532
[INFO] dual_resnorm: 0.000325
[INFO] primal_resnorm: 0.000036
[INFO] convergence: 0.000325
-------------------------------------------------
[INFO] ADMM step #91
[INFO] Updating theta...
[INFO] Newton converged in: 0.09848499298095703.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.1533350944519043.
[INFO] Updating u...
[INFO] max mu :  0.9999998442182539
[INFO] dual_resnorm: 0.000312
[INFO] primal_resnorm: 0.000034
[INF

In [543]:
theta_plot_dir

'../../output/ergm1/results4/est_theta_diff_sim1000.png'

In [544]:
print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)

plot_theta(
    theta, 
    output_dir=theta_plot_dir,
    fig_title="",
    thr=1.7,
    change_pts=sim_args['true_change_points'],
)

Saving estimated parameters...
The estimated change points are [39 59 60]


In [545]:
pd.DataFrame(theta).head()

,0,1,2,3
0,-1.992392,-0.555816,1.540198,0.512615
1,-1.992392,-0.555816,1.540198,0.512615
2,-1.992400,-0.555896,1.540200,0.512618
3,-1.992363,-0.555840,1.540201,0.512620
4,-1.992354,-0.555820,1.540200,0.512620


# ergm2

terms:
* edges + triangle + nodematch("Sex")
* n: 30
* T: 100
* n cpt: 4



In [89]:
%run ../lib/mple_learn_lib.ipynb
%run ../lib/report_lib.ipynb
%run ../lib/make_data_lib.ipynb

In [90]:
sex = pd.read_table('../../data/ergm/sex.txt', delimiter=' ').values.squeeze()

In [136]:
%%time
conf = {
    "outdir": "../../output/ergm2/results4", # update this for each new method
    "data_name": "sim2000.rds",
#     "form_terms": ['edges'],
#     "diss_terms": ['edges'],
#     "form_terms": ["nodematch('Sex')"],
#     "diss_terms": ["nodematch('Sex')"],
#     "form_terms": [ 'triangle'],
#     "diss_terms": [ 'triangle'],
#     "form_terms": ['edges', 'triangle', 'transitiveties'],
#     "diss_terms": ['edges', 'triangle', 'transitiveties'],
    "form_terms": ['edges', 'transitiveties'],
    "diss_terms": ['edges', 'transitiveties'],
    "true_change_points": [ 24, 49, 74],
}

input_data = conf["data_name"] + ('' if conf["data_name"].endswith('.rds') else '.rds')
data = readRDS("../../data/ergm/" + input_data)

output_dir = conf["outdir"] + ('' if conf["outdir"].endswith('/') else '/')
H_outfile = output_dir + conf["data_name"] + "_H.txt"
y_outfile = output_dir + conf["data_name"] + "_y.txt"


args_dir = utils.make_directory(output_dir, 'args')
utils.save_args(conf, args_dir + "args.txt")
save_H_y(data[0], conf, H_outfile, y_outfile, exogenous=sex)

time series length: 100
network size: 30 x 30
statistics dimension: 4
form terms are: ['edges', 'transitiveties']
diss terms are: ['edges', 'transitiveties']
Saving change statistics...
shape of H: (100, 900, 4)
shape of y: (100, 900)
CPU times: user 1.92 s, sys: 50.1 ms, total: 1.97 s
Wall time: 1.97 s


In [137]:
# args = {
#     'inputdir': "../../output/ergm3",
#     "result_identifier": 1,
# }

H, y, experiment_dir, sim_args = load_data(conf)
# result_dir = make_directory(experiment_dir, 'results')
# args_dir = make_directory(result_dir, 'args' + str(args["result_identifier"]))
# save_args(args, args_dir + 'args_model.txt')
theta_outfile = experiment_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = experiment_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = experiment_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = experiment_dir + 'est_theta_diff_' + sim_args["data_name"][:-4] + '.png'

INFO:root:Loading network statistics and edge data...
INFO:root:Data has dimension (t, n, p): (100, 30, 4)


{"outdir": "../../output/ergm2/results4", "data_name": "sim2000.rds", "form_terms": ["edges", "transitiveties"], "diss_terms": ["edges", "transitiveties"], "true_change_points": [24, 49, 74]}



In [145]:
conf = {
    'lam': 2,
    'admm_alpha': 100,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 50,
    'newton_max_steps': 100,
    'converge_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
    "verbose": 1,
}

In [146]:
%%time
model = STERGMGraph(
    **conf,
)

model.load_H_y(H, y)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(
    solver=conf["solver"],
#     initial_values={
#         "theta": np.tile(np.array([-1.111, -1.982, -2.057, 2.590]), (model.t,1)),
#         "u": np.zeros((model.t, model.p)),
#         "z": np.random.normal(loc=0, scale=0.2, size=(model.t, model.p)),
#     }
)
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 0.7269377708435059.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.010824203491210938.
[INFO] Updating u...
[INFO] max mu :  0.9844470710822882
[INFO] dual_resnorm: 0.359526
[INFO] primal_resnorm: 0.384097
[INFO] convergence: 0.384097
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 0.6978402137756348.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.012643098831176758.
[INFO] Updating u...
[INFO] max mu :  0.9729474815509118
[INFO] dual_resnorm: 0.249719
[INFO] primal_resnorm: 0.168389
[INFO] convergence: 0.249719
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 0.15038013458251953.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.010547876358032227.
[INFO] Updating u...
[INFO] max mu :  0.82612343

[INFO] Newton converged in: 0.13918089866638184.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.039235830307006836.
[INFO] Updating u...
[INFO] max mu :  0.8705783392881715
[INFO] dual_resnorm: 0.019303
[INFO] primal_resnorm: 0.025533
[INFO] convergence: 0.025533
-------------------------------------------------
[INFO] ADMM step #24
[INFO] Updating theta...
[INFO] Newton converged in: 0.14008188247680664.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0367889404296875.
[INFO] Updating u...
[INFO] max mu :  0.8706807327904975
[INFO] dual_resnorm: 0.019549
[INFO] primal_resnorm: 0.024771
[INFO] convergence: 0.024771
-------------------------------------------------
[INFO] ADMM step #25
[INFO] Updating theta...
[INFO] Newton converged in: 0.14073491096496582.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.03605008125305176.
[INFO] Updating u...
[INFO] max mu :  0.8707896079590617
[INFO] dual_resnorm: 0.019794
[INFO] primal_resnorm: 0.024016
[IN

[INFO] Group fused Lasso converged in: 0.05304288864135742.
[INFO] Updating u...
[INFO] max mu :  0.8725747696143923
[INFO] dual_resnorm: 0.027787
[INFO] primal_resnorm: 0.010000
[INFO] convergence: 0.027787
-------------------------------------------------
[INFO] ADMM step #46
[INFO] Updating theta...
[INFO] Newton converged in: 0.14889192581176758.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.06518387794494629.
[INFO] Updating u...
[INFO] max mu :  0.872634905799614
[INFO] dual_resnorm: 0.028352
[INFO] primal_resnorm: 0.009850
[INFO] convergence: 0.028352
-------------------------------------------------
[INFO] ADMM step #47
[INFO] Updating theta...
[INFO] Newton converged in: 0.15265893936157227.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.07015395164489746.
[INFO] Updating u...
[INFO] max mu :  0.872867618928823
[INFO] dual_resnorm: 0.028667
[INFO] primal_resnorm: 0.008651
[INFO] convergence: 0.028667
----------------------------------------------

In [147]:
theta_plot_dir

'../../output/ergm2/results4/est_theta_diff_sim2000.png'

In [148]:
H.max()

31.0

In [149]:
print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)

plot_theta(
    theta, 
    output_dir=theta_plot_dir,
    fig_title="",
    thr=1,
    change_pts=sim_args['true_change_points'],
)

Saving estimated parameters...
The estimated change points are [49 74]


In [150]:
pd.DataFrame(theta).head(25)

,0,1,2,3
0,0.687018,-2.635082,1.655682,-0.161247
1,0.687018,-2.635082,1.655682,-0.161247
2,0.686007,-2.636974,1.655600,-0.161052
3,0.685494,-2.637739,1.655567,-0.161025
4,0.685558,-2.637626,1.655529,-0.160921
5,0.686541,-2.636038,1.655528,-0.160894
6,0.686542,-2.636200,1.655466,-0.161190
7,0.685360,-2.637433,1.655545,-0.161124
8,0.685463,-2.637474,1.655533,-0.160827
9,0.685135,-2.638121,1.655451,-0.160854


In [151]:
theta = np.loadtxt('../../output/mit1/results1/theta_mit.txt')

In [163]:
np.where(norm_diff > 0.03)

(array([ 37,  38,  93, 177, 199]),)